# Jinja
一個架網站的環境，不是語言(跟html一個概念)。[原始網站](https://jinja.palletsprojects.com/en/3.1.x/)
1. html只能在template裡面使用。
2. css、js、所有要連結的圖片、影片、音樂只能在static裡面使用。
3. 有些地方只能用單引號 -> 盡量用單引號寫。
4. 基本語法：
    1. `{{ ... }}` 輸出寫法
    
        e.g.href="{{url_for('static',filename='css/bootstrap.css')}}" -> 將static/css/bootstrap.css裡的css設定輸出給該html。

    2. `{% ... %}` 描述語法
    
        e.g.{% extends "base.jinja.html" %} -> 描述：繼承base.jinja.html的所有語法
    
    3. `{# ... #}`：註釋語法

## 引入
通常一般的引入方式(相對路徑/絕對路徑)會失敗，要改個方法。
### 引入CSS
語法：假設我的css存在static/css裡。

`<link rel="stylesheet" href="{{url_for('static',filename='css/css名稱.css')}}"> `

範例：在html打

`<link rel="stylesheet" href="{{url_for('static',filename='css/style.css')}}"> `

或是`href="/static/css/style.css"`，但不推薦用這個。這樣以後如果突然想改放在不同資料夾裡的效果還要去查路徑、改，讓程式去找比較實在。

### 引入JS
語法：假設我的js存在static/js裡。

`<script src="{{url_for('static',filename='js/js名稱.js')}}"></script>`

範例：在html，body最下面打
`<script src="{{url_for('static',filename='js/bootstrap.js')}}"></script>`

### 在html、css裡引入圖片
#### html
語法：假設我的圖片存在static/img裡。

`<img src="{{url_for('static',filename='imh/圖片名稱.jpg')}}" alt="">`

#### css
就是普通的引入方式。

`background-image:url(../img/圖片名稱.jpg);`

### 把網頁b連到網頁a裡
語法：

`<p><a href="{{url_for('網頁b的def名')}}">超連結名稱</a></p>`

範例：

網頁b：
```
@app.route("/test")
def b():
    return "<h1>bbbb</h1>"
```

在網頁a的html裡打：

`<p><a href="{{url_for('b')}}">bbb</a></p>`

## 模板設計
可以少打很多程式的各種方法。

### 加入參數
在.py設定特定參數，之後在打html時就不用重複打。

#### 範例1-加入參數name

```
from flask import render_template

@app.route("/<name>")
def a(name):
    return "Hi, {name}."
```

`<name>`：輸入參數的地方 -> 如果要讓這個網站能順利開啟，就要打`網址/想輸入的名字`。範例：http://127.0.0.1:5000/Steven

`def a(name):`：讓這個函數能輸入name這個參數。

`"Hi, {name}."`：會自動轉換成輸入的name值 -> Hi, Steven.

![jinja1](./image/jinja1.jpg)

#### 範例2-在html裡加入參數name
```
from flask import render_template

@app.route("/<name>")
def a(name):
    return render_template('a.html',name = name)
```

`name=name`：在html裡的{{name}}會自動轉換成輸入的name值 -> Steven

![jinja2](./image/jinja2.jpg)

#### 進階-**locals()
`**locals()`：傳遞所有參數與區域變數。

1. 設定引入值

    .py
    ```
    from flask import render_template

    @app.route("/<name>")
    def hellohello(name):
        data={
            "name":name,
            "age":18,
            "location":"Taiwan",
        }
        return render_template('test-f.html',**locals())
    ```

    `data={"name":name,"age":18,"location":"Taiwan",}`：創建一個(可以引入的)dict。

    `"name":name`："name"這個key裡面的值是`<name>`輸入參數的值。

2. 引入：dict名稱.key名稱

    .html的body

    `111{{data.name}} {{data.age}} {{data.location}}`

    ![jinja3](./image/jinja3.jpg)

### {% for ... in ... %} ... {% endfor %}
用途：Loop(加上else的python語法)。

#### 範例1：重複列點
在.py設定網站的地方引入a這個list

```
@app.route("/")
def main():
    a=["aaa","bbb","ccc"]
    return render_template('main.jinja.html',a=a)
```

1. a=["aaa","bbb","ccc"] 在上面打a的值 

2. a=a 把a引入 

3. 在html輸入

    ```
    {%for i in a%}
        <li>
            {{i}}
        </li>
    {%endfor%}
    ```
    網頁就會顯示

    + aaa
    + bbb
    + ccc

#### 範例2
# 這裡還沒寫完

### 繼承-{% extends "base.jinja.html" %}
繼承所有base裡的html設定 -> 不用重複打

#### 流程 - 範本製作
1. 創一個base.html(不能用別的名字)
2. 一般地創建網頁(! ↲)、建架構
3. 其他會不同的地方用以下程式省略。

範例：

我的`<div class="content"></div>`裡面的內容要不同：

在base.html：

```
<div class="content">
{%block content%}{%endblock%}
</div>
```

### 修改標題 - {%block title%}{%endblock%}
1. 方法1：全部重打
    + 在base.html：`<title>{%block title%}{%endblock%}</title>`
    + 在其他的網頁：`{%block title%}網頁名稱 - 分頁別{%endblock%}`

        範例：`{%block title%}Apple - a{%endblock%}`

2. 方法2：只改部分
    + 在base.html：`<title>{%block title%}網頁名稱 - {%endblock%}</title>`

        範例：`<title>{%block title%}Apple - {%endblock%}</title>`

    + 在其他的網頁：`{%block title%}分頁別{%endblock%}`

        範例：`{%block title%}a{%endblock%}`

### 修改head - {%block head%}{%endblock%}
目前可用武之地在多掛自己的css時。

範例：我要在apple.html裡多掛apple.css

+ 在base.html：
    ```
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{%block title%}Apple - {%endblock%}</title>
        <link rel="stylesheet" href="{{url_for('static',filename='css/css名稱.css')}}">
        {%block head%}{%endblock%}
    </head>
    ```
+ 在其他的網頁：`{%block head%}<link rel="stylesheet" href="{{url_for('static',filename='css/css名稱.css')}}">{%endblock%}`

        範例：`{%block head%}<link rel="stylesheet" href="{{url_for('static',filename='css/apple.css')}}">{%endblock%}`

### 修改內容 - {%block content%}{%endblock%}
最常見的使用方式。

範例：

我的`<div class="content"></div>`裡面的內容要不同：

+ 在base.html：

    ```
    <div class="content">
    {%block content%}{%endblock%}
    </div>
    ```

+ 在其他的網頁：{%block content%}內文{%endblock%}